In [1]:
print("hello world")

hello world


In [ ]:
# ============================================================
# SMARTVISION AI - RESNET18 TRAINING (COLAB + PYTORCH + GPU)
# ============================================================

# -----------------------------
# Mount Google Drive
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')

# -----------------------------
# Imports
# -----------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from PIL import Image
import matplotlib.pyplot as plt
import os

# -----------------------------
# Device
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------
# Dataset Paths (GOOGLE DRIVE)
# -----------------------------
BASE_PROJECT_DIR = "/content/drive/MyDrive/SmartVision AI - Intelligent Multi-Class Object Recognition System"
DATASET_DIR = f"{BASE_PROJECT_DIR}/smartvision_dataset_v5/classification"

train_dir = f"{DATASET_DIR}/train"
val_dir   = f"{DATASET_DIR}/val"
test_dir  = f"{DATASET_DIR}/test"

single_prediction_image = f"{DATASET_DIR}/test/dog/dog_00012.jpg"

# -----------------------------
# Model Save Directory (PERSISTENT)
# -----------------------------
models_dir = f"{BASE_PROJECT_DIR}/models"
os.makedirs(models_dir, exist_ok=True)

best_model_path = f"{models_dir}/best_resnet18_smartvision_cropped.pth"

print("\nModel will be saved to:")
print(best_model_path)

# -----------------------------
# Transforms (ResNet Standard)
# -----------------------------
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# -----------------------------
# Datasets
# -----------------------------
train_dataset = ImageFolder(train_dir, transform=train_transform)
val_dataset   = ImageFolder(val_dir,   transform=val_test_transform)
test_dataset  = ImageFolder(test_dir,  transform=val_test_transform)

# -----------------------------
# DataLoaders (OPTIMIZED FOR COLAB)
# -----------------------------
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"\nTraining samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# -----------------------------
# Class Info
# -----------------------------
class_list = train_dataset.classes
num_classes = len(class_list)

print(f"\nNumber of classes: {num_classes}")
for i, cls in enumerate(class_list):
    print(f"{i:2d}: {cls}")

# -----------------------------
# Load ResNet18 (Transfer Learning)
# -----------------------------
model = models.resnet18(weights="IMAGENET1K_V1")

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# -----------------------------
# Loss, Optimizer, Scheduler
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Mixed precision (FAST)
scaler = torch.cuda.amp.GradScaler()

# -----------------------------
# Training Loop
# -----------------------------
num_epochs = 15
best_val_acc = 0.0

print("\n" + "="*80)
print("STARTING TRAINING (RESNET18 | COLAB GPU)")
print("="*80)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    train_acc = 100. * correct / total
    train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_total += labels.size(0)
            val_correct += preds.eq(labels).sum().item()

    val_acc = 100. * val_correct / val_total
    val_loss /= len(val_loader)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"\n>>> BEST MODEL SAVED ({val_acc:.2f}%)")

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.2f}%")

    scheduler.step()

print("\nTRAINING COMPLETED")
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"Model saved at:\n{best_model_path}")

# -----------------------------
# Test Evaluation
# -----------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

test_acc = 100. * correct / total
print(f"\nFINAL TEST ACCURACY: {test_acc:.2f}%")

# -----------------------------
# Single Image Prediction
# -----------------------------
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    input_tensor = val_test_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1)
        conf, idx = torch.max(probs, 1)

    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.title(f"{class_list[idx.item()].upper()} ({conf.item()*100:.2f}%)")
    plt.axis("off")
    plt.show()

predict_image(single_prediction_image)

print("\nALL DONE ✅")
# ============================================================
# SMARTVISION AI - RESNET18 TRAINING (COLAB + PYTORCH + GPU)
# ============================================================

# -----------------------------
# Mount Google Drive
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')

# -----------------------------
# Imports
# -----------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from PIL import Image
import matplotlib.pyplot as plt
import os

# -----------------------------
# Device
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------
# Dataset Paths (GOOGLE DRIVE)
# -----------------------------
BASE_PROJECT_DIR = "/content/drive/MyDrive/SmartVision AI - Intelligent Multi-Class Object Recognition System"
DATASET_DIR = f"{BASE_PROJECT_DIR}/smartvision_dataset_v5/classification"

train_dir = f"{DATASET_DIR}/train"
val_dir   = f"{DATASET_DIR}/val"
test_dir  = f"{DATASET_DIR}/test"

single_prediction_image = f"{DATASET_DIR}/test/dog/dog_00012.jpg"

# -----------------------------
# Model Save Directory (PERSISTENT)
# -----------------------------
models_dir = f"{BASE_PROJECT_DIR}/models"
os.makedirs(models_dir, exist_ok=True)

best_model_path = f"{models_dir}/best_resnet18_smartvision_cropped.pth"

print("\nModel will be saved to:")
print(best_model_path)

# -----------------------------
# Transforms (ResNet Standard)
# -----------------------------
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# -----------------------------
# Datasets
# -----------------------------
train_dataset = ImageFolder(train_dir, transform=train_transform)
val_dataset   = ImageFolder(val_dir,   transform=val_test_transform)
test_dataset  = ImageFolder(test_dir,  transform=val_test_transform)

# -----------------------------
# DataLoaders (OPTIMIZED FOR COLAB)
# -----------------------------
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"\nTraining samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# -----------------------------
# Class Info
# -----------------------------
class_list = train_dataset.classes
num_classes = len(class_list)

print(f"\nNumber of classes: {num_classes}")
for i, cls in enumerate(class_list):
    print(f"{i:2d}: {cls}")

# -----------------------------
# Load ResNet18 (Transfer Learning)
# -----------------------------
model = models.resnet18(weights="IMAGENET1K_V1")

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# -----------------------------
# Loss, Optimizer, Scheduler
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Mixed precision (FAST)
scaler = torch.cuda.amp.GradScaler()

# -----------------------------
# Training Loop
# -----------------------------
num_epochs = 15
best_val_acc = 0.0

print("\n" + "="*80)
print("STARTING TRAINING (RESNET18 | COLAB GPU)")
print("="*80)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    train_acc = 100. * correct / total
    train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_total += labels.size(0)
            val_correct += preds.eq(labels).sum().item()

    val_acc = 100. * val_correct / val_total
    val_loss /= len(val_loader)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"\n>>> BEST MODEL SAVED ({val_acc:.2f}%)")

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.2f}%")

    scheduler.step()

print("\nTRAINING COMPLETED")
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"Model saved at:\n{best_model_path}")

# -----------------------------
# Test Evaluation
# -----------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

test_acc = 100. * correct / total
print(f"\nFINAL TEST ACCURACY: {test_acc:.2f}%")

# -----------------------------
# Single Image Prediction
# -----------------------------
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    input_tensor = val_test_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1)
        conf, idx = torch.max(probs, 1)

    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.title(f"{class_list[idx.item()].upper()} ({conf.item()*100:.2f}%)")
    plt.axis("off")
    plt.show()

predict_image(single_prediction_image)

print("\nALL DONE ✅")


Using device: cuda
Models will be saved to:
   E:\Desktop\GUVI\Project\SmartVision AI - Intelligent Multi-Class Object Recognition System\models
Best model path:
   E:\Desktop\GUVI\Project\SmartVision AI - Intelligent Multi-Class Object Recognition System\models/best_resnet18_smartvision_cropped.pth



FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Desktop\\GUVI\\Project\\SmartVision AI - Intelligent Multi-Class Object Recognition System\\smartvision_dataset_v4_2500\\classification/train'